In [0]:
%sql
use catalog setorfarmaceutico;

In [0]:
%sql
DESCRIBE DETAIL setorfarmaceutico.bronze.vendas_produtos_finais;

In [0]:
df = spark.table("setorfarmaceutico.bronze.vendas_produtos_finais")
df.show()


In [0]:
df.printSchema()

In [0]:
from pyspark.sql.functions import col, to_date, count, mean, datediff, when

df = (df
    .withColumn("nr_nf", col("nr_nf").cast("int"))
    .withColumn("nr_linha", col("nr_linha").cast("int"))
    .withColumn("quantidade_vendida", col("quantidade_vendida").cast("int"))
    .withColumn("valor_unitario", col("valor_unitario").cast("float"))
    .withColumn("valor_total", col("valor_total").cast("float"))
    .withColumn("imposto_percentual", col("imposto_percentual").cast("float"))
    .withColumn("imposto_reais", col("imposto_reais").cast("float"))
    .withColumn("data_emissao_nf", to_date(col("data_emissao_nf"), "yyyy-MM-dd"))
    .withColumn("data_vencimento", to_date(col("data_vencimento"), "yyyy-MM-dd"))
)

In [0]:
tipo_clientes_counts = df.groupBy("tipo_cliente").count()
tipo_clientes_counts.show()

##### A base está extremamente equilibrada entre 4 segmentos.
- Isso permite análises por canal e estratégias de venda específicas.

In [0]:
produto_top = (
    df.groupBy("nome_produto")
      .agg(count("*").alias("qtd"))
      .orderBy(col("qtd").desc())
      .limit(10)
)
produto_top.show(truncate=False)

##### Esses produtos aparecem muito, mas o insight mais importante é o faturamento (acima).

In [0]:
from pyspark.sql.functions import sum as spark_sum
valor = df.agg(spark_sum(col("valor_total")).alias("faturamento")).collect()[0]["faturamento"]
valor_formatado = f"{valor:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")  # formatação PT-BR
print("Faturamento total:", f"R$ {valor_formatado}")


##### Faturamento total: R$ 9.708.593.780,39 (9,7 bilhões)

In [0]:
faturamento_por_estado = (
    df.groupBy("estado")
      .agg(spark_sum(col("valor_total")).alias("faturamento"))
      .orderBy(col("faturamento").desc())
      .limit(10)
)
faturamento_por_estado.show(truncate=False)

##### Os estados com maior volume de vendas são também os com maior faturamento.
- Regiões Nordeste e Sul aparecem forte.

In [0]:
valor = df.agg(mean(col("valor_total")).alias("ticket_medio")).collect()[0]["ticket_medio"]
ticket_formatado = f"{valor:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
print("Ticket médio: R$ " + ticket_formatado)
print("\nVendas grandes, de alto valor unitário.")

In [0]:
imposto_medio = df.agg(mean(col("imposto_percentual")).alias("imposto_medio"))
imposto_medio.show()

##### Percentual está dentro do esperado para produtos farmacêuticos (12.50%)

In [0]:
produto_mais_lucrativo = (
    df.groupBy("nome_produto")
      .agg(spark_sum(col("valor_total")).cast("decimal(20,2)").alias("faturamento"))
      .orderBy(col("faturamento").desc())
      .limit(10)
)
produto_mais_lucrativo.show(truncate=False)

##### Os produtos “343”, “352” e “24” são os campeões de faturamento.

In [0]:
df = df.withColumn("atraso_dias", datediff(col("data_vencimento"), col("data_emissao_nf")))
atraso_medio = df.agg(mean(col("atraso_dias")).alias("atraso_medio"))
atraso_medio.show()

##### Prazo médio de pagamento de 2 meses.

In [0]:
# Margem bruta estimada (sem custo real)
df = (df.withColumn("margem_bruta", col("valor_total") - col("imposto_reais"))
.withColumn("margem_percentual", col("margem_bruta") / col("valor_total")))
# Útil para ranking de produtos mais rentáveis

In [0]:
# Prazo de pagamento real (dias)
df = df.withColumn("prazo_dias", datediff(col("data_vencimento"), col("data_emissao_nf")))
# Pode gerar KPIs por estado, cliente, tipo de cliente.

In [0]:
# Valor médio por unidade vendida
df = df.withColumn("valor_medio_por_unidade",col("valor_total") / col("quantidade_vendida"))
# Permite identificar produtos premium.

In [0]:
# Classificação de portes de venda
df = df.withColumn(
    "porte_venda",
    when(col("valor_total") <= 5000, "Pequena")
    .when((col("valor_total") > 5000) & (col("valor_total") <= 20000), "Média")
    .when((col("valor_total") > 20000) & (col("valor_total") <= 50000), "Grande")
    .when(col("valor_total") > 50000, "Gigante")
)
# Útil para entender distribuição de vendas.

In [0]:
# Imposto efetivo aplicado (Dá para detectar notas com impostos fora do padrão):
df = df.withColumn(
    "imposto_efetivo_pct",
    (col("imposto_reais") / col("valor_total")) * 100)

In [0]:
df.limit(5).show()

In [0]:
%sql
use schema silver;

In [0]:
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver.stg_vendas_produtos_finais")


#### Resumo
- 380 mil vendas registradas.

- Ticket médio alto: R$ 12,5 mil por venda.

- Faturamento total superior a R$ R$ 9.708.593.780,39

- Tipos de clientes equilibrados (farmácia, hospital, SUS, outros).

- Produtos “343”, “352” e “24” geram maior receita.

- Imposto médio aplicado: 12,5%.

- Estados com maior participação: PE, GO, CE, PR, BA.

- Prazo médio de pagamento: 60 dias.

- Métrica "Margem Bruta" -> Mede o lucro antes de custos operacionais

- Métrica "Prazo de Pagamento" -> Permite análises de ciclo financeiro e risco de recebíveis.

- Métrica "Valor Médio por Unidade" -> Classifica produtos entre premium x baixo custo.

- Métrica "Porte de Venda" -> Ótimo para clusterização comercial e perfil de clientes
  - ClassificaçÃo do tamanho das vendas:
    - Pequena
    - Média
    - Grande
    - Gigante
  - Usando faixas de valor_total

- Métrica "Imposto Efetivo (%)" -> Detecta notas com imposto fora do padrão.
  